In [2]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

sm_boto3=boto3.client("sagemaker")
sess=sagemaker.Session()
region=sess.boto_session.region_name
bucket='taskbucket532'
print("Using bucket",bucket)


sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\lokeshsinha\AppData\Local\sagemaker\sagemaker\config.yaml
Using bucket taskbucket532


### Load Dataset

In [3]:
data=pd.read_csv("seattle-weather.csv")
data.head()

,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


### Data Modelling

In [4]:
data=data.iloc[:,1:]
data.head()

,precipitation,temp_max,temp_min,wind,weather
0,0.0,12.8,5.0,4.7,drizzle
1,10.9,10.6,2.8,4.5,rain
2,0.8,11.7,7.2,2.3,rain
3,20.3,12.2,5.6,4.7,rain
4,1.3,8.9,2.8,6.1,rain


In [16]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
data["weather_encoded"]=labelencoder.fit_transform(data["weather"])
data.head(20)

,precipitation,temp_max,temp_min,wind,weather,weather_encoded
0,0.0,12.8,5.0,4.7,drizzle,0
1,10.9,10.6,2.8,4.5,rain,2
2,0.8,11.7,7.2,2.3,rain,2
3,20.3,12.2,5.6,4.7,rain,2
4,1.3,8.9,2.8,6.1,rain,2
5,2.5,4.4,2.2,2.2,rain,2
6,0.0,7.2,2.8,2.3,rain,2
7,0.0,10.0,2.8,2.0,sun,4
8,4.3,9.4,5.0,3.4,rain,2
9,1.0,6.1,0.6,3.4,rain,2


In [6]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
x=data[['precipitation','temp_max','temp_min','wind']]
y=data['weather_encoded']

x_train,x_test, y_train,y_test=train_test_split(x,y,test_size=0.3)

### Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logistic_reg=LogisticRegression()
logistic_reg.fit(x_train,y_train)
x_pred1=logistic_reg.predict(x_test)
accuracy_log=accuracy_score(y_test,x_pred1)
accuracy_log

c:\Users\lokeshsinha\OneDrive\Desktop\#Projects\MLops\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8496583143507973

### Decision Tree

In [8]:
from sklearn.tree import DecisionTreeClassifier

decision_tree= DecisionTreeClassifier()
decision_tree.fit(x_train,y_train)

x_pred2=decision_tree.predict(x_test)
accuracy_dec=accuracy_score(y_test,x_pred2)
accuracy_dec

0.7494305239179955

### Support Vector Machine

In [9]:
from sklearn.svm import SVC
svc=SVC(kernel='linear')
svc.fit(x_train,y_train)

svc_predict=svc.predict(x_test)
accuracy_svc=accuracy_score(y_test,svc_predict)
accuracy_svc


0.8473804100227791

### Random Forest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier
random_forest=RandomForestClassifier()
random_forest.fit(x_train,y_train)

random_forest_pred=random_forest.predict(x_test)
accuracy_rf=accuracy_score(random_forest_pred,y_test)
accuracy_rf

0.8314350797266514

## Pickel Model


In [14]:
import pickle
pickle.dump(logistic_reg,open('./model1.sav','wb'))
pickle.dump(decision_tree,open('./model2.sav','wb'))
pickle.dump(svc,open('./model3.sav','wb'))
pickle.dump(random_forest,open('./model4.sav','wb'))